# Toy model of upper glycolysis

An example of how to use modelbase v.1.2.
We replicated here the toy model of upper glycolysis and used it to show the mca class for the Metabolic Control Analysis. 

This notebook replicates the results of the toy model introduced for an educational purpose in the Systems Biology textbook by [Edda Klipp et al.](https://books.google.de/books?id=V1ZiEw5eo7EC&printsec=frontcover&hl=pl#v=onepage&q&f=false), results from p. 87)

# Model construction
## Import packages

In [ ]:
import numpy as np
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

from modelbase.ode import Model, LabelModel, LinearLabelModel, Simulator, mca
from modelbase.ode import ratefunctions as rf

## define parameters and compounds

In [ ]:
parameters = {
    "k1": 0.25,
    "k2": 1,
    "k3": 1,
    "k3m": 1,
    "k4": 1,
    "k5": 1,
    "k6": 1,
    "k7": 2.5,
}

compounds = {"GLC", "G6P", "F6P", "FBP", "ATP", "ADP"}

## Instantiate model

In [ ]:
m = Model(parameters, compounds)

# add reactions using defined ratefunctions
m.add_reaction(
    rate_name="v1",
    function=rf.constant,
    stoichiometry={"GLC": 1},
    parameters=["k1"],
)
m.add_reaction(
    rate_name="v2",
    function=rf.mass_action_2,
    stoichiometry={"GLC": -1, "ATP": -1, "G6P": 1, "ADP": 1},
    parameters=["k2"],
)
m.add_reaction(
    rate_name="v3",
    function=rf.reversible_mass_action_1_1,
    stoichiometry={"G6P": -1, "F6P": 1},
    parameters=["k3", "k3m"],
    reversible=True,
)
m.add_reaction(
    rate_name="v4",
    function=rf.mass_action_2,
    stoichiometry={"F6P": -1, "ATP": -1, "ADP": 1, "FBP": 1},
    parameters=["k4"],
)
m.add_reaction(
    rate_name="v5",
    function=rf.mass_action_1,
    stoichiometry={"FBP": -1, "F6P": 1},
    parameters=["k5"],
)
m.add_reaction(
    rate_name="v6",
    function=rf.mass_action_1,
    stoichiometry={"FBP": -1},
    parameters=["k6"],
)
m.add_reaction(
    rate_name="v7",
    function=rf.mass_action_1,
    stoichiometry={"ADP": -1, "ATP": 1},
    parameters=["k7"],
)

In [ ]:
m.get_stoichiometric_df()

# Dynamic behaviour of the system

In [ ]:
y0 = {
    "GLC": 0,
    "G6P": 0,
    "F6P": 0,
    "FBP": 0,
    "ATP": 0.5,
    "ADP": 0.5,
}

s = Simulator(m)
s.initialise(y0)
t, y = s.simulate(t_end=60, steps=10000)
fig, ax = s.plot(xlabel="time [au]", ylabel="concentration [au]")
plt.show()

# Flux and concentration control coefficients

In [ ]:
parameters = m.get_parameters().keys()

crc = mca.get_concentration_response_coefficients_df(
    model=m, parameters=parameters, y=y0, normalized=True
).loc[:, ["GLC", "G6P", "F6P", "FBP", "ATP", "ADP"]]

frc = mca.get_flux_response_coefficients_df(
    model=m, parameters=parameters, y=y0, normalized=True
)

In [ ]:
fig, axs = mca.plot_multiple(
    dfs=[crc, frc],
    titles=["Concentration Response Coefficients", "Flux Response Coefficients"],
)